<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Build Interactive LLM Agents with Tools**


----


### Installing Required Libraries


In [ ]:
%pip install langchain===0.3.25 | tail -n 1
%pip install langchain-openai===0.3.19 | tail -n 1

### Importing Required Libraries
Recommendation:Import all required libraries in one place (here):_


In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage

Let's initialize the language model that will power your tool calling capabilities. This code sets up a GPT-4o-mini model using the OpenAI provider through LangChain's interface, which you'll use to process queries and decide which tools to call.


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
# IGNORE IF YOU ARE NOT RUNNING LOCALLY
from langchain_openai import ChatOpenAI
from langchain_ibm import ChatWatsonx
openai_llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key = "your openai api key here",
)
watsonx_llm = ChatWatsonx(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="your project id associated with the API key",
    api_key="your watsonx.ai api key here",
)

## Building an Agent

In [ ]:
class ToolCallingAgent:
    def __init__(self, llm):
        self.llm_with_tools = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        # Step 1: Initial user message
        chat_history = [HumanMessage(content=query)]

        # Step 2: LLM chooses tool
        response = self.llm_with_tools.invoke(chat_history)
        if not response.tool_calls:
            return response.contet # Direct response, no tool needed
        # Step 3: Handle first tool call
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_call_id = tool_call["id"]

        # Step 4: Call tool manually
        tool_result = self.tool_map[tool_name].invoke(tool_args)

        # Step 5: Send result back to LLM
        tool_message = ToolMessage(content=str(tool_result), tool_call_id=tool_call_id)
        chat_history.extend([response, tool_message])

        # Step 6: Final LLM result
        final_response = self.llm_with_tools.invoke(chat_history)
        return final_response.content

## Exercises


### Exercise 1: Create a new tool

Use the example tool format provided in the notebook to create a new tool named `calculate_tip` that takes a `total_bill and tip_percent`, and returns the tip amount. </br>
Define and invoke the tool with sample inputs like `total_bill=120`, `tip_percent=15`. </br>
Create a `tool_map` with the `calculate_tip` tool.


In [ ]:
@tool
def calculate_tip(total_bill: int, tip_percent: int) -> int:
    """Calculate tip"""
    return total_bill * tip_percent * 0.01

inputs = {
    "total_bill": 120,
    "tip_percent": 15
}
calculate_tip.invoke(inputs)

tool_map = {
    "calculate_tip": calculate_tip
}

### Exercise 2: Tool calling with an LLM

Simulate a user query like "How much should I tip on $60 at 20%?". </br>
Bind the tool to the predefined `llm` and prompt the LLM with the query above. Then parse the LLM response for the tool calling details and invoke the tool accordingly. Finally, take the entire chat history and prompt the LLM for a final output.


In [ ]:
query = "How much should I tip on $60 at 20%?"

llm_with_tool = llm.bind_tools([calculate_tip])
chat_history = [HumanMessage(content=query)]

response = llm_with_tool.invoke(chat_history)

tool_calls = response.tool_calls
tool_name = tool_calls[0]["name"]
tool_args = tool_calls[0]["args"]
tool_call_id = tool_calls[0]["id"]

tool_response = tool_map[tool_name].invoke(tool_args)
tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_id)

chat_history.extend([response, tool_message])

result = llm_with_tool.invoke(chat_history)
print(result.content)

### Exercise 3: Create a tip calculating agent

Create an agent to automate the entire process you previously completed.


In [ ]:
class TipAgent:
    def __init__(self, llm):
        self.llm_with_tool = llm.bind_tools([calculate_tip])
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        chat_history = [HumanMessage(content=query)]
        response = llm_with_tool.invoke(chat_history)

        tool_calls = response.tool_calls
        tool_name = tool_calls[0]["name"]
        tool_args = tool_calls[0]["args"]
        tool_call_id = tool_calls[0]["id"]

        tool_response = tool_map[tool_name].invoke(tool_args)
        tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_id)

        chat_history.extend([response, tool_message])

        return llm_with_tool.invoke(chat_history).content

agent = TipAgent(llm)
query = "How much should I tip on $60 at 20%?"
agent.run(query)

## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>


<!-- ## Changelog

| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-06-06 | 0.1 |  P. Kravitz | ID review and edit. No code edits.Updated the copyright statement. Change log added. Instructional edits only for IBM style. Second person, accessibility, and other minor grammar edits.| -->
